In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.10.0-dev20220717


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-07-17 16:44:46--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.006s  

2022-07-17 16:44:46 (56.3 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2022-07-17 16:44:46--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[========

In [ ]:
train_dataset = pd.read_csv(train_file_path, delimiter = '\t', 
                            names = ['label','text'])
test_dataset = pd.read_csv(test_file_path, delimiter = '\t', 
                           names = ['label','text'])

EDA

In [ ]:
train_dataset

,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [ ]:
test_dataset

,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
train_dataset.isna().sum()

label    0
text     0
dtype: int64

In [ ]:
test_dataset.isna().sum()

label    0
text     0
dtype: int64

In [ ]:
train_dataset['label'].value_counts()

ham     3619
spam     560
Name: label, dtype: int64

In [ ]:
test_dataset['label'].value_counts()

ham     1205
spam     187
Name: label, dtype: int64

Data is unbalanced for the given two classes

We over-sample them to bring balance

In [ ]:
def over_sample(df, output):
    '''
    Over samples that class which have lesser data-points
    '''

    count_0, count_1 = df[output].value_counts()
    df_0 = df[df[output]=='ham']
    df_1 = df[df[output]=='spam']

    if count_0 > count_1 :
        df_1_over = df_1.sample(count_0, replace = True)
        df_test_over = pd.concat([df_0, df_1_over])
    elif count_0 < count_1:
        df_0_over = df_0.sample(count_1, replace = True)
        df_test_over = pd.concat([df_0_over,df_1])

    return df_test_over

In [ ]:
train_dataset = over_sample(train_dataset, 'label')
test_dataset = over_sample(test_dataset, 'label')

In [ ]:
train_dataset['label'] = train_dataset['label'].map({'ham':0, 'spam':1})
test_dataset['label'] = test_dataset['label'].map({'ham':0, 'spam':1})

In [ ]:
train_text = train_dataset.copy()
test_text = test_dataset.copy()

train_label = train_text.pop('label')
test_label = test_text.pop('label')

In [ ]:
train_text

,text
0,ahhhh...just woken up!had a bad dream about u ...
1,you can never do nothing
2,"now u sound like manky scouse boy steve,like! ..."
3,mum say we wan to go then go... then she can s...
4,never y lei... i v lazy... got wat? dat day ü ...
...,...
2801,save money on wedding lingerie at www.bridal.p...
4154,for ur chance to win a £250 cash every wk txt:...
3744,splashmobile: choose from 1000s of gr8 tones e...
1345,urgent! this is the 2nd attempt to contact u!u...


In [ ]:
train_label

0       0
1       0
2       0
3       0
4       0
       ..
2801    1
4154    1
3744    1
1345    1
3377    1
Name: label, Length: 7238, dtype: int64

In [ ]:
train_label.value_counts()

0    3619
1    3619
Name: label, dtype: int64

In [ ]:
test_label.value_counts()

0    1205
1    1205
Name: label, dtype: int64

Since DNN doesnot accept textual data so we 

1. tokenize all words from given sentences

2. assign index to all unique words

3. convert sentences to sequences by mapping the lists to dictionary of indices

4. pad the sequences in order to give them similar shape

In [ ]:
def replace_special_char(text):
    special_characters = ['!','"','#','$','%','&','(',')',
                          '*','+','/',':',';','<','=','>','@',
                          '[','\\',']','^','`','{','|','}','~',
                          '\t','.','-']
    for i in special_characters:
        text = text.replace(i, " ")
    return text

In [ ]:
def word_tokenizer(df):
    token = []
    for i in range(df.shape[0]):
        tokens = []
        sentence = replace_special_char(df.iloc[i][0])
        for item in sentence.split(" "):
            if item != "":
                tokens.append(item)
        token.append(tokens)
    return token

In [ ]:
train_tokens = word_tokenizer(train_text)
test_tokens = word_tokenizer(test_text)

In [ ]:
train_tokens

[['ahhhh',
  'just',
  'woken',
  'up',
  'had',
  'a',
  'bad',
  'dream',
  'about',
  'u',
  'tho,so',
  'i',
  'dont',
  'like',
  'u',
  'right',
  'now',
  'i',
  'didnt',
  'know',
  'anything',
  'about',
  'comedy',
  'night',
  'but',
  'i',
  'guess',
  'im',
  'up',
  'for',
  'it'],
 ['you', 'can', 'never', 'do', 'nothing'],
 ['now',
  'u',
  'sound',
  'like',
  'manky',
  'scouse',
  'boy',
  'steve,like',
  'i',
  'is',
  'travelling',
  'on',
  'da',
  'bus',
  'home',
  'wot',
  'has',
  'u',
  'inmind',
  '4',
  'recreation',
  'dis',
  'eve?'],
 ['mum',
  'say',
  'we',
  'wan',
  'to',
  'go',
  'then',
  'go',
  'then',
  'she',
  'can',
  'shun',
  'bian',
  'watch',
  'da',
  'glass',
  'exhibition'],
 ['never',
  'y',
  'lei',
  'i',
  'v',
  'lazy',
  'got',
  'wat?',
  'dat',
  'day',
  'ü',
  'send',
  'me',
  'da',
  'url',
  'cant',
  'work',
  'one'],
 ['in',
  'xam',
  'hall',
  'boy',
  'asked',
  'girl',
  'tell',
  'me',
  'the',
  'starting',
  'term

In [ ]:
def word_index(token):
    word_index = {}
    token_flattened = [item for ele in token for item in ele]
    d = list(set(token_flattened))
    for i in range(len(d)):
        word_index[d[i]] = i+1
    return word_index

In [ ]:
train_index = word_index(train_tokens)

In [ ]:
train_index

{'worse,': 1,
 'kicks': 2,
 '07808': 3,
 'endless': 4,
 'painting?': 5,
 "can't,": 6,
 "riley's": 7,
 'rv': 8,
 '80082': 9,
 'jamster': 10,
 'bookedthe': 11,
 'spontaneously': 12,
 'by?': 13,
 'rolled': 14,
 '08002888812': 15,
 'comin': 16,
 'yaxxx': 17,
 'reminder': 18,
 '7,': 19,
 'wife,': 20,
 'discussed': 21,
 'boobs': 22,
 'subscriber': 23,
 'xy': 24,
 'throwin': 25,
 'stripes': 26,
 "he's": 27,
 'truffles': 28,
 '8007': 29,
 'rd': 30,
 'twat': 31,
 'stands,': 32,
 'beta': 33,
 "mum's": 34,
 'main': 35,
 '7876150ppm': 36,
 'month': 37,
 'delay': 38,
 'referin': 39,
 "shade's": 40,
 'home,': 41,
 'yr': 42,
 'goss': 43,
 'arrival': 44,
 '1b6a5ecef91ff9': 45,
 'cn': 46,
 'sochte': 47,
 'jsut': 48,
 'becaus': 49,
 'hillsborough': 50,
 'footprints': 51,
 'minute': 52,
 'understanding': 53,
 'inform': 54,
 'manky': 55,
 'holder,': 56,
 'wait': 57,
 "i'll": 58,
 'biro': 59,
 'ralphs': 60,
 'noe': 61,
 'hand,': 62,
 'bathing': 63,
 'dependents': 64,
 '08704050406': 65,
 'shining': 66,
 'i

In [ ]:
test_index

{'voice': 1,
 'attention': 2,
 'temp': 3,
 'misundrstud': 4,
 'statements': 5,
 'cutting': 6,
 'wan2': 7,
 'bt': 8,
 'gurl': 9,
 'bought\x94braindance\x94a': 10,
 'points': 11,
 'platt': 12,
 'constant': 13,
 'uncles': 14,
 'enemy': 15,
 'none': 16,
 'much,': 17,
 'sory': 18,
 'runs': 19,
 'w1j6hl': 20,
 'jen': 21,
 'dbuk': 22,
 'relation': 23,
 'also,': 24,
 'reach': 25,
 '01223585334': 26,
 'aiyah': 27,
 'bid': 28,
 'comin': 29,
 'saucy': 30,
 'crucify': 31,
 'konw': 32,
 'insects': 33,
 'wt': 34,
 'airport': 35,
 'otherwise': 36,
 '7,': 37,
 'ok,': 38,
 'normal': 39,
 'college?xx': 40,
 'discussed': 41,
 'late': 42,
 'tenants': 43,
 'outage': 44,
 'journey': 45,
 'xy': 46,
 'anna': 47,
 'compass': 48,
 'drink': 49,
 'owe': 50,
 'shoot': 51,
 'fifth': 52,
 'apparently': 53,
 'fats': 54,
 '02073162414': 55,
 'popping': 56,
 "he's": 57,
 '2ez': 58,
 'spin': 59,
 '8th': 60,
 '8007': 61,
 'icon': 62,
 'rd': 63,
 'annoncement': 64,
 'calicut?': 65,
 'shoving': 66,
 'pa': 67,
 'mk45': 68,


In [ ]:
def max_length(token = []):
    max = len(token[0])
    for element in token:
        new = len(element)
        if new > max:
            max = new
    return max

In [ ]:
def text_to_seq_and_padding(token = []):
    for element in token:
        for i in range(len(element)):
            try:
                element[i] = train_index[element[i]]
            except:
                element[i] = 0

    maxlen = max_length(token)

    for element in token:
        length = len(element)
        if length < maxlen:
            need = maxlen - length
            for i in range(need):
                element.append(0)    

    return np.array(token)

In [ ]:
train_padded = text_to_seq_and_padding(train_tokens)

In [ ]:
train_padded.shape

(7238, 189)

In [ ]:
test_padded = text_to_seq_and_padding(test_tokens)

In [ ]:
train_padded.shape

(7238, 189)

DNN Model for classification

In [ ]:
model = tf.keras.Sequential([
                             keras.layers.Embedding(7238, 16),
                             keras.layers.GlobalAveragePooling1D(),
                             keras.layers.Dense(24, activation = 'relu'),
                             keras.layers.Dropout(0.2),
                             keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 16)          115808    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 116,241
Trainable params: 116,241
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_padded, 
                    train_label, 
                    epochs = 30, 
                    validation_data = (test_padded, test_label), 
                    verbose = 2)

Epoch 1/30
227/227 - 2s - loss: 0.6759 - accuracy: 0.6301 - val_loss: 0.6133 - val_accuracy: 0.9332 - 2s/epoch - 8ms/step
Epoch 2/30
227/227 - 1s - loss: 0.4775 - accuracy: 0.9130 - val_loss: 0.2433 - val_accuracy: 0.9448 - 1s/epoch - 5ms/step
Epoch 3/30
227/227 - 1s - loss: 0.1900 - accuracy: 0.9632 - val_loss: 0.1468 - val_accuracy: 0.9469 - 974ms/epoch - 4ms/step
Epoch 4/30
227/227 - 1s - loss: 0.1002 - accuracy: 0.9780 - val_loss: 0.1545 - val_accuracy: 0.9365 - 952ms/epoch - 4ms/step
Epoch 5/30
227/227 - 1s - loss: 0.0676 - accuracy: 0.9816 - val_loss: 0.1330 - val_accuracy: 0.9510 - 959ms/epoch - 4ms/step
Epoch 6/30
227/227 - 1s - loss: 0.0496 - accuracy: 0.9865 - val_loss: 0.1297 - val_accuracy: 0.9548 - 948ms/epoch - 4ms/step
Epoch 7/30
227/227 - 1s - loss: 0.0386 - accuracy: 0.9906 - val_loss: 0.1353 - val_accuracy: 0.9581 - 939ms/epoch - 4ms/step
Epoch 8/30
227/227 - 1s - loss: 0.0296 - accuracy: 0.9943 - val_loss: 0.1317 - val_accuracy: 0.9593 - 919ms/epoch - 4ms/step
Epoch 

In [ ]:
model.evaluate(test_padded, test_label)

76/76 [==============================] - 0s 2ms/step - loss: 0.2185 - accuracy: 0.9643


[0.21845531463623047, 0.9643153548240662]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    test = pd.DataFrame(data = [pred_text])
    test = word_tokenizer(test)
    padded = text_to_seq_and_padding(test)
    pred = model.predict(padded)[0]
    predd = np.where(pred > 0.5, 'spam', 'ham')[0]

    prediction = [list(pred)[0], predd]
    prediction

    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 50ms/step
[1.0153639e-08, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 19ms/step
You passed the challenge. Great job!
